In [12]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import matplotlib as mpl
import mne
import pathlib
import pytorch_lightning as pl
import torch
import torcheeg
import xgboost
import wandb
import autoreject
from tqdm.notebook import tqdm

In [13]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../src/utils/')
from transforms import _compose, _randomcrop, totensor, \
channelwide_norm, channelwise_norm, _clamp, toimshape, \
_labelcenter, _labelnorm, _labelbin, _labelmultibin
from functools import partial
sys.path.append('../drafts/schwarz/')

from data import BrainAgeDataset

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

### Interface with data

In [15]:
data_dir = pathlib.Path("/data0/practical-sose23/brain-age/data/bap/raw/")
assert data_dir.is_dir()

data_dir_healthy = data_dir / "healthy_controls/preprocessed"
assert data_dir_healthy.is_dir()

data_dir_chronic_pain = data_dir / "chronic_pain_patients"
assert data_dir_chronic_pain.is_dir()

In [16]:
eeg_files_all = list(data_dir.rglob("*.vhdr"))

eeg_files_raw = [f for f in eeg_files_all if not "preprocessed" in str(f)]
eeg_files_preprocessed = [f for f in eeg_files_all if "preprocessed" in str(f)]

eeg_files_healthy = list(data_dir_healthy.rglob("*.vhdr"))
eeg_files_chronic_pain = [f for f in eeg_files_preprocessed if "chronic_pain_patients" in str(f)]


### Analyze the subject data

In [17]:
def get_subj_id(eeg_file: pathlib.Path):
    return eeg_file.stem.split("_")[-3]

def get_age(df_subj, eeg_file: pathlib.Path):
    subj_id = get_subj_id(eeg_file)
    return df_subj[df_subj["Subject ID"] == subj_id]["Age(years)"].values[0]

#### Load clinical data

In [18]:
f_subj = data_dir / "clinical_data_updated_2020-08-04.ods"
assert f_subj.is_file()

In [19]:
df_subj_chronic_pain = pd.read_excel(f_subj, sheet_name=0, engine="odf", skiprows=1)
df_subj_healthy = pd.read_excel(f_subj, sheet_name=1, engine="odf", skiprows=0)

df_subj = pd.concat([df_subj_chronic_pain, df_subj_healthy])
df_subj["Age(years)"] = df_subj["Age(years)"].fillna(df_subj["Age (years)"])

print(f"# recorded subjects:      {len(df_subj)}")
print(f"# raw eeg files:          {len(eeg_files_raw)}")
print(f"# preprocessed eeg files: {len(eeg_files_preprocessed)}")

zfill_ints = lambda x:str(x).zfill(3) if type(x) else x
df_subj_chronic_pain["Subject ID"] = df_subj_chronic_pain["Subject ID"].astype(str).map(zfill_ints)
df_subj_healthy["Subject ID"] = df_subj_healthy["Subject ID"].astype(str).map(zfill_ints)
df_subj["Subject ID"] = df_subj["Subject ID"].astype(str).map(zfill_ints)

df_subj

# recorded subjects:      189
# raw eeg files:          190
# preprocessed eeg files: 189


,Subject ID,Project prefix,Age(years),Sex (m/f),Pain duration(months),Avg. painIntensity (0 – 10),Curr. Pain Intensity(0 – 10),BDI,PDQ,MQS,VR-12 PCS,VR-12 MCS,PDI,Age (years)
0,VP02,chronic_back_pain,67.0,m,360.0,5.0,5.0,3.0,28.0,4.0,NaN,NaN,NaN,NaN
1,VP04,chronic_back_pain,54.0,f,120.0,5.0,4.0,35.0,14.0,31.7,NaN,NaN,NaN,NaN
2,VP05,chronic_back_pain,64.0,f,96.0,7.0,5.0,7.0,16.0,11.4,NaN,NaN,NaN,NaN
3,VP07,chronic_back_pain,41.0,m,22.0,6.0,7.0,15.0,22.0,6.1,NaN,NaN,NaN,NaN
4,VP08,chronic_back_pain,72.0,m,480.0,8.0,8.0,11.0,10.0,8.4,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,vpH64,various,79.0,f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,79.0
84,vpH66,various,56.0,f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,56.0
85,vpH67,various,54.0,m,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54.0
86,vpH68,various,55.0,f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55.0


#### Check the metadata and set channel types

In [20]:
example_raw = mne.io.read_raw_brainvision(eeg_files_chronic_pain[0])
example_raw

Extracting parameters from /data0/practical-sose23/brain-age/data/bap/raw/chronic_pain_patients/chronic_back_pain/preprocessed/Exp_eyes_closed_vp11_eyes_closed.vhdr...
Setting channel info structure...


/tmp/ipykernel_148081/2177493425.py:1: RuntimeWarning: No coordinate information found for channels ['Fp1', 'Fp2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2', 'F7', 'F8', 'T7', 'T8', 'P7', 'P8', 'Fz', 'Cz', 'Pz', 'Oz', 'FC1', 'FC2', 'CP1', 'CP2', 'FC5', 'FC6', 'CP5', 'CP6', 'TP9', 'TP10', 'LE', 'RE', 'P1', 'P2', 'C1', 'C2', 'FT9', 'FT10', 'AF3', 'AF4', 'FC3', 'FC4', 'CP3', 'CP4', 'PO3', 'PO4', 'F5', 'F6', 'C5', 'C6', 'P5', 'P6', 'PO9', 'Iz', 'FT7', 'FT8', 'TP7', 'TP8', 'PO7', 'PO8', 'Fpz', 'PO10', 'CPz', 'POz']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  example_raw = mne.io.read_raw_brainvision(eeg_files_chronic_pain[0])
/tmp/ipykernel_148081/2177493425.py:1: RuntimeWarning: Not setting positions of 64 misc channels found in montage:
['Fp1', 'Fp2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2', 'F7', 'F8', 'T7', 'T8', 'P7', 'P8', 'Fz', 'Cz', 'Pz', 'Oz', 'FC1', 'FC2', 'CP1', 'CP2', 'FC5', 'FC6', 'CP5', 'CP6', 'TP9', 'TP10', 'LE', 'RE', 'P

<RawBrainVision | Exp_eyes_closed_vp11_eyes_closed.dat, 65 x 300004 (300.0 s), ~71 kB, data not loaded>

In [21]:
channel_to_channel_type = {ch_name:"eeg" for ch_name in example_raw.ch_names}
channel_to_channel_type.update({"LE":"misc", "RE":"misc"})
eeg_chs = [ch for ch in example_raw.ch_names if not (ch=="RE" or ch=="LE")]
example_raw.set_channel_types(channel_to_channel_type)


/tmp/ipykernel_148081/179859254.py:4: RuntimeWarning: The unit for channel(s) AF3, AF4, C1, C2, C3, C4, C5, C6, CP1, CP2, CP3, CP4, CP5, CP6, CPz, Cz, F3, F4, F5, F6, F7, F8, FC1, FC2, FC3, FC4, FC5, FC6, FT10, FT7, FT8, FT9, Fp1, Fp2, Fpz, Fz, Iz, O1, O2, Oz, P1, P2, P3, P4, P5, P6, P7, P8, PO10, PO3, PO4, PO7, PO8, PO9, POz, Pz, T7, T8, TP10, TP7, TP8, TP9 has changed from NA to V.
  example_raw.set_channel_types(channel_to_channel_type)


<RawBrainVision | Exp_eyes_closed_vp11_eyes_closed.dat, 65 x 300004 (300.0 s), ~71 kB, data not loaded>

In [22]:
montage = mne.channels.make_standard_montage("standard_1020")
example_raw = example_raw.set_montage(montage)

In [23]:
channel_to_channel_type = {ch_name:"eeg" for ch_name in example_raw.ch_names}
channel_to_channel_type.update({"LE":"misc", "RE":"misc"})

In [24]:
for f in eeg_files_preprocessed:
    if not f.is_file():
        print(f)

### Preprocess data

In [ ]:
sfreq = 100 #int(example_raw.info["sfreq"])
epoch_len = int(sfreq * 30)
l_freq = 0.1
h_freq = 45 #sfreq/(2+1)

epochs = []
ages = []

for f in tqdm(eeg_files_preprocessed):
    
    sub_id = get_subj_id(f)
    age = get_age(df_subj, f)
    
    raw = mne.io.read_raw_brainvision(f, verbose=False, preload=True)
    raw.set_channel_types(channel_to_channel_type)
    raw.set_montage(montage)
    
    raw = raw.filter(l_freq=l_freq, h_freq=h_freq, h_trans_bandwidth=5)
    raw = raw.resample(sfreq=sfreq)
    
    epochs_subj = mne.make_fixed_length_epochs(raw, duration=30, preload=True)
    epochs.append(epochs_subj.get_data())
    ages.append(len(epochs_subj)*[age])


  0%|          | 0/189 [00:00<?, ?it/s]

/tmp/ipykernel_148081/3760871422.py:14: RuntimeWarning: No coordinate information found for channels ['Fp1', 'Fp2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2', 'F7', 'F8', 'T7', 'T8', 'P7', 'P8', 'Fz', 'Cz', 'Pz', 'Oz', 'FC1', 'FC2', 'CP1', 'CP2', 'FC5', 'FC6', 'CP5', 'CP6', 'TP9', 'TP10', 'LE', 'RE', 'P1', 'P2', 'C1', 'C2', 'FT9', 'FT10', 'AF3', 'AF4', 'FC3', 'FC4', 'CP3', 'CP4', 'PO3', 'PO4', 'F5', 'F6', 'C5', 'C6', 'P5', 'P6', 'PO9', 'Iz', 'FT7', 'FT8', 'TP7', 'TP8', 'PO7', 'PO8', 'Fpz', 'PO10', 'CPz', 'POz']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  raw = mne.io.read_raw_brainvision(f, verbose=False, preload=True)
/tmp/ipykernel_148081/3760871422.py:14: RuntimeWarning: Not setting positions of 64 misc channels found in montage:
['Fp1', 'Fp2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2', 'F7', 'F8', 'T7', 'T8', 'P7', 'P8', 'Fz', 'Cz', 'Pz', 'Oz', 'FC1', 'FC2', 'CP1', 'CP2', 'FC5', 'FC6', 'CP5', 'CP6', 'TP9', 'TP10', 'LE', 'RE', 'P1

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 5.00 Hz (-6 dB cutoff frequency: 47.50 Hz)
- Filter length: 33001 samples (33.001 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  63 out of  63 | elapsed:    0.7s finished


Not setting metadata
10 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 10 events and 3000 original time points ...
1 bad epochs dropped


...

/tmp/ipykernel_148081/3760871422.py:14: RuntimeWarning: Not setting positions of 64 misc channels found in montage:
['Fp1', 'Fp2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2', 'F7', 'F8', 'T7', 'T8', 'P7', 'P8', 'Fz', 'Cz', 'Pz', 'Oz', 'FC1', 'FC2', 'CP1', 'CP2', 'FC5', 'FC6', 'CP5', 'CP6', 'TP9', 'TP10', 'LE', 'RE', 'P1', 'P2', 'C1', 'C2', 'FT9', 'FT10', 'AF3', 'AF4', 'FC3', 'FC4', 'CP3', 'CP4', 'PO3', 'PO4', 'F5', 'F6', 'C5', 'C6', 'P5', 'P6', 'PO9', 'Iz', 'FT7', 'FT8', 'TP7', 'TP8', 'PO7', 'PO8', 'Fpz', 'PO10', 'CPz', 'POz']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  raw = mne.io.read_raw_brainvision(f, verbose=False, preload=True)
/tmp/ipykernel_148081/3760871422.py:15: RuntimeWarning: The unit for channel(s) AF3, AF4, C1, C2, C3, C4, C5, C6, CP1, CP2, CP3, CP4, CP5, CP6, CPz, Cz, F3, F4, F5, F6, F7, F8, FC1, FC2, FC3, FC4, FC5, FC6, FT10, F

Not setting metadata
10 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 10 events and 3000 original time points ...
6 bad epochs dropped


/tmp/ipykernel_148081/3760871422.py:14: RuntimeWarning: No coordinate information found for channels ['Fp1', 'Fp2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2', 'F7', 'F8', 'T7', 'T8', 'P7', 'P8', 'Fz', 'Cz', 'Pz', 'Oz', 'FC1', 'FC2', 'CP1', 'CP2', 'FC5', 'FC6', 'CP5', 'CP6', 'TP9', 'TP10', 'LE', 'RE', 'P1', 'P2', 'C1', 'C2', 'FT9', 'FT10', 'AF3', 'AF4', 'FC3', 'FC4', 'CP3', 'CP4', 'PO3', 'PO4', 'F5', 'F6', 'C5', 'C6', 'P5', 'P6', 'PO9', 'Iz', 'FT7', 'FT8', 'TP7', 'TP8', 'PO7', 'PO8', 'Fpz', 'PO10', 'CPz', 'POz']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  raw = mne.io.read_raw_brainvision(f, verbose=False, preload=True)


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 5.00 Hz (-6 dB cutoff frequency: 47.50 Hz)
- Filter length: 33001 samples (33.001 sec)



/tmp/ipykernel_148081/3760871422.py:14: RuntimeWarning: Not setting positions of 64 misc channels found in montage:
['Fp1', 'Fp2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2', 'F7', 'F8', 'T7', 'T8', 'P7', 'P8', 'Fz', 'Cz', 'Pz', 'Oz', 'FC1', 'FC2', 'CP1', 'CP2', 'FC5', 'FC6', 'CP5', 'CP6', 'TP9', 'TP10', 'LE', 'RE', 'P1', 'P2', 'C1', 'C2', 'FT9', 'FT10', 'AF3', 'AF4', 'FC3', 'FC4', 'CP3', 'CP4', 'PO3', 'PO4', 'F5', 'F6', 'C5', 'C6', 'P5', 'P6', 'PO9', 'Iz', 'FT7', 'FT8', 'TP7', 'TP8', 'PO7', 'PO8', 'Fpz', 'PO10', 'CPz', 'POz']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  raw = mne.io.read_raw_brainvision(f, verbose=False, preload=True)
/tmp/ipykernel_148081/3760871422.py:15: RuntimeWarning: The unit for channel(s) AF3, AF4, C1, C2, C3, C4, C5, C6, CP1, CP2, CP3, CP4, CP5, CP6, CPz, Cz, F3, F4, F5, F6, F7, F8, FC1, FC2, FC3, FC4, FC5, FC6, FT10, F

### Neural network baseline

In [ ]:
from torcheeg.models import EEGNet

In [ ]:
hparams_eegnet = {
    "learning_rate":1e-3,
    "batch_size":128,
    "chunk_size":int(sfreq*4),
    "dropout":0.2,
    "kernel_1": int(sfreq//2),
    "kernel_2": int(sfreq//8),
    "F1":16,
    "F2":32,
    "depth_multiplier":2
}

eegnet = EEGNet(chunk_size=hparams_eegnet["chunk_size"],
               num_electrodes=63,
               dropout=hparams_eegnet["dropout"],
               kernel_1=hparams_eegnet["kernel_1"],
               kernel_2=hparams_eegnet["kernel_2"],
               F1=hparams_eegnet["F1"],
               F2=hparams_eegnet["F2"],
               D=hparams_eegnet["depth_multiplier"],
               num_classes=1)

#### Split data across subjects

In [ ]:
np.random.seed(42), np.random.shuffle(epochs)
np.random.seed(42), np.random.shuffle(ages)

n_train = int(0.7*len(epochs))
n_val = int(0.15*len(epochs))

epochs_train = epochs[:n_train]
ages_train = ages[:n_train]

epochs_val = epochs[n_train:n_train+n_val]
ages_val = ages[n_train:n_train+n_val]

In [ ]:
epochs_train = np.concatenate(epochs_train)
ages_train = np.concatenate(ages_train)
epochs_val = np.concatenate(epochs_val)
ages_val = np.concatenate(ages_val)

#### Configure transforms

In [ ]:
mean_age = torch.tensor(round(ages_train.mean(), 3))

randomcrop = partial(_randomcrop, seq_len=hparams_eegnet["chunk_size"])
clamp = partial(_clamp, dev_val=20.0)
labelcenter = partial(_labelcenter, mean_age=round(df_subj["Age(years)"].mean(), 3))
labelbin = partial(_labelbin, y_lower=mean_age)
transforms = partial(_compose, transforms=[totensor,toimshape, randomcrop, channelwise_norm, clamp])
target_transforms = partial(_compose, transforms=[labelcenter, totensor])

In [ ]:
dataset_train = BrainAgeDataset(
    epochs_train, 
    ages_train, 
    transforms=transforms, 
    target_transforms=target_transforms
)
dataset_val = BrainAgeDataset(
    epochs_val, 
    ages_val, 
    transforms=transforms, 
    target_transforms=target_transforms
)


dataloader_train = torch.utils.data.DataLoader(dataset_train, batch_size=hparams_eegnet["batch_size"], num_workers=8, shuffle=True, drop_last=True)
dataloader_val = torch.utils.data.DataLoader(dataset_val, batch_size=hparams_eegnet["batch_size"], num_workers=8, drop_last=True)

print(f"# training samples: {len(dataset_train)}")
print(f"# validation samples: {len(dataset_val)}")

#### Visualize random data points 

In [ ]:
for eeg, age in dataloader_train.dataset:
    plt.figure(figsize=(12,5))
    print(eeg.mean(), eeg.std(), eeg.max())
    plt.title(f"age label: {str(age.item())}")
    plt.imshow(eeg[0], aspect="auto")
    
    plt.plot(eeg[0].T + torch.arange(eeg.shape[1]).unsqueeze(0), "r", alpha=0.25)
    plt.xlabel(f"time [@{sfreq}Hz]")

    plt.show()
    break



In [ ]:
# len(np.concatenate(epochs)), len(np.concatenate(ages)), \
# len(np.concatenate(epochs_train)), len(np.concatenate(ages_train))

In [ ]:
class BrainAgeModel(pl.LightningModule):
        
    def __init__(self, model, hparams, loss_func, metric=None):
        super().__init__()
        self.model = model
        self.hparams.update(hparams)
        self.loss_func = loss_func
        self.metric = metric
        
    def forward(self, x):
        x = self.model(x)
        return x
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.hparams["learning_rate"])
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        val_loss = self.loss_func(y_hat.squeeze(), y.squeeze())
        self.log("validation loss", val_loss, on_step=True, on_epoch=True, prog_bar=True)
        if self.metric:
            metric_val = self.metric["func"](y_hat.squeeze(), y.squeeze())
#             print(f"==================Val Acc: {metric_val} =======================")
#             print(y_hat.squeeze()[:2], y.squeeze()[:2])
            self.log("val_"+self.metric["name"], metric_val, on_step=True, on_epoch=True, prog_bar=True)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        loss = self.loss_func(y_hat.squeeze(), y.squeeze())
        self.log("training loss", loss, on_step=True, on_epoch=True, prog_bar=True)
        if self.metric:
            metric_val = self.metric["func"](y_hat.squeeze(), y.squeeze())
#             print(f"==================Train Acc: {metric_val} =======================")
#             print(y_hat.squeeze()[:2], y.squeeze()[:2])
            self.log("train_"+self.metric["name"], metric_val, on_step=True, on_epoch=True, prog_bar=True)

        return loss

#### Set-up metrics

In [ ]:
def score_accuracy(y_hat, y):
    return (y==torch.argmax(y_hat, dim=-1)).float().mean()

def score_binary_accuracy(y_hat, y):
    y_hat = torch.sigmoid(y_hat)
    return (y==torch.round(y_hat)).float().mean()

def score_binary_accuracy_v2(y_hat, y):
    y_hat = torch.sigmoid(y_hat)
    return (y==torch.where(y_hat > 0.5, 1.0, 0.0)).float().mean()

def _score_r2(y_hat, y, y_var):
    return  1 - torch.nn.functional.mse_loss(y.squeeze(), y_hat.squeeze()) / y_var
    
score_r2 = partial(_score_r2, y_var=df_subj["Age(years)"].var())

def compute_r2(model, val_dataloader):
    model.eval(), model.cpu()
    rss, tss = 0, 0
    for batch in val_dataloader:
        x, y = batch
        y_hat = model.forward(x)
        rss += torch.nn.functional.mse_loss(y.squeeze(), y_hat.squeeze())
        tss += y.var()
    return 1 - rss/tss

### Initialize model

In [ ]:
model = BrainAgeModel(
    model=list(eegnet.modules())[0], 
    hparams=hparams_eegnet, 
    loss_func=torch.nn.functional.l1_loss,
    metric={"name":"r2", "func":score_r2}
)
print(model)

#### Test fwd pass

In [ ]:
# example_out = model(dataloader_train.dataset[0][0].unsqueeze(0))
example_out = model(next(iter(dataloader_train))[0])
example_out.shape, example_out.min(), example_out.max()

#### Specify experiment name and train 

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
wandb.finish()

overfit_batches = 1.0
wandb.login()
logger = pl.loggers.WandbLogger(project="brain-age-ssl", name=f"EEGNet regression baseline on {overfit_batches} batches", 
                                save_dir="/data0/practical-sose23/brain-age", log_model=False)

early_stop_callback = EarlyStopping(monitor="validation loss", min_delta=0.00, patience=50, verbose=False, mode="min")

trainer = pl.Trainer(
    callbacks=[early_stop_callback], 
    max_epochs=500, 
    accelerator="gpu",
    precision="bf16",
    logger=logger)
# Tuner(trainer).lr_find(model, dataloader_train)

trainer.fit(model, dataloader_train, dataloader_val)
wandb.finish()

In [ ]:
!nvidia-smi